In [ ]:
import pandas as pd
import os
import mysql.connector

# List of CSV file paths
csv_files = [
    "andhra_bus_details.csv", "assam_bus_details.csv", "chandigarh_bus_details.csv", 
    "himachal_bus_details.csv", "kadamba_bus_details.csv", "kerala_bus_details.csv",
    "rajasthan_bus_details.csv", "Telangana_bus_details.csv", 
    "up_bus_details.csv", "wb2_bus_details.csv"
]

# Check for missing files
missing_files = [file for file in csv_files if not os.path.exists(file)]
if missing_files:
    print(f"Missing files: {', '.join(missing_files)}")

# Read only the existing files
existing_files = [file for file in csv_files if os.path.exists(file)]
df_list = [pd.read_csv(file) for file in existing_files]

# Concatenate all DataFrames in the list
combined_df = pd.concat(df_list, ignore_index=True)

# Add ID column
combined_df['id'] = range(1, len(combined_df) + 1)

# Data Cleaning
combined_df['Price'] = combined_df['Price'].fillna('').str.replace('INR ', '')
combined_df['Seat_Availability'] = combined_df['Seat_Availability'].str.extract(r'(\d+)').fillna(0)

# MySQL connection
myconnection = mysql.connector.connect(
    host='localhost', user='root', passwd='123456789', database="redbus_data"  # Ensure correct database
)
cursor = myconnection.cursor()

# Define service_code for dynamic table creation
service_code = "apsrtc"  # Set the service code dynamically
table_name = f"{service_code}_routes"

# Create the service-specific table if it doesn't exist
try:
    cursor.execute(f"SELECT 1 FROM {table_name} LIMIT 1")  # Check if table exists
except mysql.connector.Error as err:
    if err.errno == mysql.connector.errorcode.ER_NO_SUCH_TABLE:
        print(f"Table {table_name} does not exist. Creating it...")
        # Create the table (adjust schema as needed)
        cursor.execute(f"""
        CREATE TABLE {apsrtc} (
            route_id INT AUTO_INCREMENT PRIMARY KEY,
            route_name VARCHAR(255),
            route_link VARCHAR(255)
        );
        """)
        print(f"Table {table_name} created successfully.")
    else:
        print(f"Error: {err}")
        raise

# Generate MySQL column definitions from DataFrame's dtypes for the bus_routes table
columns = []
for col, dtype in zip(combined_df.columns, combined_df.dtypes):
    mysql_dtype = 'TEXT' if dtype == 'object' else ('INT' if dtype == 'int64' else 'FLOAT')
    columns.append(f"`{col}` {mysql_dtype}")
column_definition = ", ".join(columns)

# Create the main bus_routes table if it doesn't exist
cursor.execute(f"CREATE TABLE IF NOT EXISTS bus_routes ({column_definition})")

# Insert data into the bus_routes table (batch insert for performance)
insert_query = f"INSERT INTO bus_routes ({', '.join(combined_df.columns)}) VALUES (%s, " + ", ".join(["%s"] * (len(combined_df.columns) - 1)) + ")"
cursor.executemany(insert_query, combined_df.values.tolist())

# Commit the changes to bus_routes table
myconnection.commit()

# Ensure the service-specific data gets inserted
insert_service_query = f"INSERT INTO {table_name} (route_name, route_link) VALUES (%s, %s)"
routes_data = combined_df[['route_name', 'route_link']].values.tolist()
cursor.executemany(insert_service_query, routes_data)

# Commit the changes to the service-specific table
myconnection.commit()

# Close connection
cursor.close()
myconnection.close()

# Save the combined DataFrame to a CSV file
combined_df.to_csv("bus_routes.csv", index=False)

print("Data inserted successfully and saved to 'bus_routes.csv'.")
